In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from operator import add

In [2]:
df1 = pd.read_csv('Dataset.data', delim_whitespace=True)

In [3]:
col = ['Sex', 'Length', 'Diameter', 'Height', 'Whole', 'Shucked', 'Viscera', 'Shell', 'Rings']
df1.columns = col

In [4]:
# Target Variable
y = df1['Rings'].values

In [5]:
ind_Sex = pd.get_dummies(df1['Sex'], prefix='Sex')
df2 = pd.concat([ind_Sex, df1.select_dtypes(['float64']), df1.select_dtypes(['int64'])], axis = 1)

In [6]:
for cat, value in df2.iteritems():
    if(cat != 'Rings'):
        df2[cat] = (df2[cat].values - np.mean(df2[cat].values))/np.std(df2[cat].values)

In [7]:
R_train_mod = []
R_val_mod = []
for i in range(5):
    if i == 0:
        X_train_mod = df2.loc[835*(i+1):, 'Sex_F':'Shell']
        X_train_mod.insert(0, 'Intercept', np.ones(3342), True)

        y_train_mod = df2.loc[835*(i+1):, 'Rings']
    elif i == 4:
        X_train_mod = df2.loc[:(835*i)-1, 'Sex_F':'Shell']
        X_train_mod.insert(0, 'Intercept', np.ones(3340), True)

        y_train_mod = df2.loc[:(835*i)-1, 'Rings']
    else:
        X_train_mod = pd.concat([df2.loc[:(835*i)-1, 'Sex_F':'Shell'], df2.loc[835*(i+1):, 'Sex_F':'Shell']], axis=0)
        X_train_mod.insert(0, 'Intercept', np.ones(3342), True)

        y_train_mod = pd.concat([df2.loc[:(835*i)-1, 'Rings'], df2.loc[835*(i+1):, 'Rings']], axis=0)


    if i == 4:
        X_validation_mod = df2.loc[(835*i):, 'Sex_F':'Shell']
        X_validation_mod.insert(0, 'Intercept', np.ones(837), True)

        y_validation_mod = df2.loc[(835*i):, 'Rings']
    else:
        X_validation_mod = df2.loc[(835*i):(835*(i+1)-1), 'Sex_F':'Shell']
        X_validation_mod.insert(0, 'Intercept', np.ones(835), True)

        y_validation_mod = df2.loc[(835*i):(835*(i+1)-1), 'Rings']
    
    
    theta_t_mod = (np.linalg.inv(np.dot(X_train_mod.T, X_train_mod))).dot(np.dot(X_train_mod.T, y_train_mod))
    hypo_mod = np.dot(X_train_mod, theta_t_mod)
    err = hypo_mod - y_train_mod
    RMSE_train_mod = math.sqrt((np.dot(err.T, err))/(2*len(y_train_mod)))
    print("For fold {} RMSE for training set = {}".format(i+1,RMSE_train_mod))
    R_train_mod.append(RMSE_train_mod)

    pred_mod = np.dot(X_validation_mod, theta_t_mod)
    error = pred_mod - y_validation_mod
    RMSE_val_mod = math.sqrt((np.dot(error.T, error))/(2*len(y_validation_mod)))
    print("For fold {} RMSE for validation set = {}".format(i+1,RMSE_val_mod))
    R_val_mod.append(RMSE_val_mod)
    print("---")

For fold 1 RMSE for training set = 1.3723589317275606
For fold 1 RMSE for validation set = 2.2043949202046975
---
For fold 2 RMSE for training set = 1.635668156695796
For fold 2 RMSE for validation set = 1.2264904073187626
---
For fold 3 RMSE for training set = 1.5326112900115405
For fold 3 RMSE for validation set = 1.7137666623020236
---
For fold 4 RMSE for training set = 1.5981910859526425
For fold 4 RMSE for validation set = 1.366908579260397
---
For fold 5 RMSE for training set = 1.58776318307603
For fold 5 RMSE for validation set = 1.41656211787528
---


In [8]:
# fig1 = plt.figure(1)
# plt.plot(R_train_mod)
# plt.title('RMSE Train vs Folds Plot(Normal Equation)')
# plt.xlabel('Folds')
# plt.ylabel('RMSE')
# plt.show()

In [9]:
# fig2 = plt.figure(2)
# plt.plot(R_val_mod)
# plt.title('RMSE Validation vs Folds Plot(Normal Equation)')
# plt.ylabel('RMSE')
# plt.xlabel('Folds')
# plt.show()